In [ ]:
import sys
import os
from icecream import ic

from pathlib import Path

import utils_behavior

from utils_behavior import Ballpushing_utils
from utils_behavior import Utils
from utils_behavior import Processing
from utils_behavior import HoloviewsTemplates

import pandas as pd
import numpy as np


# from sklearn.decomposition import PCA
# from sklearn.preprocessing import StandardScaler
import pandas as pd
#import pyarrow.feather as feather

import pandas as pd
import numpy as np



import matplotlib.pyplot as plt
import seaborn as sns

import importlib



In [ ]:
# Get the path to the data

Datapath = Utils.get_data_path()

In [ ]:
# Find folders with "Learning or learning" in the name as a list

folders = [f for f in Datapath.glob("*FeedingState*")]

folders

In [ ]:
# Take the first 

exp = Ballpushing_utils.Experiment(folders[0])

In [ ]:
exp

In [ ]:
Dataset = Ballpushing_utils.Dataset(exp)

In [ ]:
ball_data = Dataset.data

In [ ]:
ball_data

In [ ]:
# Now let's compute the euclidean distance between the fly and the ball during events

exfly = ball_data.loc[ball_data["fly"] == ball_data["fly"].unique()[0]]

In [ ]:
exfly["adjusted_frame"] = 0

In [ ]:
# for each event, compute the distance between the fly and the ball
for event in exfly["interaction_event"].unique():
    event_data = exfly.loc[exfly["interaction_event"] == event]
    event_data = event_data.sort_values("frame")
    event_data["distance"] = np.sqrt(
        (event_data["x_ball_0"] - event_data["x_fly_0"]) ** 2
        + (event_data["y_ball_0"] - event_data["y_fly_0"]) ** 2
    )
    
    # Add an "adjusted_frame" column that starts at 0 and goes up to the length of the event
    event_data["adjusted_frame"] = range(len(event_data))

    # Assign the computed columns back to the original DataFrame
    exfly.loc[exfly["interaction_event"] == event, "distance"] = event_data["distance"]
    exfly.loc[exfly["interaction_event"] == event, "adjusted_frame"] = event_data["adjusted_frame"]

print(exfly)

In [ ]:
exfly

In [ ]:
exfly.columns

In [ ]:
# Find first minimum distance for each event
min_distance = exfly.groupby("interaction_event")["distance"].min()

In [ ]:
# Filter out NaN values from interaction_event
exfly = exfly.dropna(subset=['interaction_event'])

# Rest of your preprocessing code...

# Find first minimum distance for each event
min_distance = exfly.groupby("interaction_event")["distance"].idxmin()

# Create a grid
grid = sns.FacetGrid(exfly, col="interaction_event", col_wrap=4)

# Plot the distance for each event
grid.map(plt.plot, "adjusted_frame", "distance")

# Add markers for the minimum distance points
for ax, event in zip(grid.axes.flat, exfly["interaction_event"].unique()):
    try:
        min_idx = min_distance[event]
        min_point = exfly.loc[min_idx]
        ax.plot(min_point["adjusted_frame"], min_point["distance"], 'ro')
    except KeyError:
        print(f"Warning: No minimum distance found for event {event}")

# Save the plot
plt.savefig("/mnt/upramdya_data/MD/MultiMazeRecorder/Plots/250218_Refined_contacts/events_distance_withmarkers_shortevents_New.png")
plt.show()



In [ ]:
type(exfly["interaction_event"].unique())

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.signal import find_peaks, savgol_filter

def find_interaction_start(data, distance_col, frame_col, 
                          threshold_multiplier=1.5, window_size=30, 
                          min_plateau_length=50, peak_prominence=1,
                          peak_window_size=10):
    # Work on a copy to avoid modifying original data
    data = data.copy()
    
    # Smoothing and derivative calculation
    data['smoothed_distance'] = savgol_filter(data[distance_col], 
                                            window_length=11, 
                                            polyorder=3)
    data['smoothed_diff'] = data['smoothed_distance'].diff()
    data['smoothed_accel'] = data['smoothed_diff'].diff()  # Second derivative

    # Dynamic threshold for plateaus based on rolling standard deviation
    rolling_std = data['smoothed_diff'].rolling(window=window_size).std()
    dynamic_threshold = rolling_std.mean() * threshold_multiplier  # Adjust multiplier as needed

    # Plateau detection with dynamic threshold
    plateau_mask = (data['smoothed_diff'].abs() < dynamic_threshold)
    plateau_groups = (plateau_mask != plateau_mask.shift()).cumsum()
    
    # Initialize plateau markers
    data['plateau_start'] = 0
    valid_plateaus = data[plateau_mask].groupby(plateau_groups).filter(
        lambda x: len(x) >= min_plateau_length
    )
    if not valid_plateaus.empty:
        start_indices = valid_plateaus.groupby(plateau_groups).head(1).index
        data.loc[start_indices, 'plateau_start'] = 1

    # Peak detection with stricter prominence
    peaks, _ = find_peaks(-data['smoothed_distance'],
                        prominence=peak_prominence,
                        width=3)
    
    # Refine peak detection for better alignment
    refined_peaks = []
    for peak in peaks:
        if peak > 0 and peak < len(data) - 1:
            # Perform local search around the detected peak to refine position
            local_region = data.iloc[max(0, peak - peak_window_size):min(len(data), peak + peak_window_size)]
            true_peak_idx = local_region[distance_col].idxmin()  # Find true minimum in this region
            refined_peaks.append(true_peak_idx)

    # Combine plateau and refined peak detections
    plateau_indices = data[data['plateau_start'] == 1].index
    all_candidates = sorted(list(plateau_indices) + refined_peaks)
    
    # Fallback to minimum distance if no markers found
    if not all_candidates:
        return data[distance_col].idxmin()
    
    return all_candidates[0]

In [ ]:

# Convert interaction_event to integer sequence first
exfly['interaction_event'] = exfly['interaction_event'].astype(int)

# Filter out NaN values from interaction_event
exfly = exfly.dropna(subset=['interaction_event'])

# Find first minimum distance and plateau/peak for each event
min_distance = exfly.groupby("interaction_event")["distance"].idxmin()
plateau_or_peak = exfly.groupby("interaction_event").apply(
    lambda x: find_interaction_start(x, 'distance', 'adjusted_frame',
                                    )
)

# Create a grid
grid = sns.FacetGrid(exfly, col="interaction_event", col_wrap=4)

# Plot the distance for each event
grid.map(plt.plot, "adjusted_frame", "distance")

# Add markers for the minimum distance points and plateau/peak points
for ax, event in zip(grid.axes.flat, exfly["interaction_event"].unique()):
    try:
        # Plot minimum distance point
        min_idx = min_distance[event]
        min_point = exfly.loc[min_idx]
        ax.plot(min_point["adjusted_frame"], min_point["distance"], 'ro', label='Minimum')
        
        # Plot plateau/peak point
        plateau_peak_idx = plateau_or_peak[event]
        plateau_peak_point = exfly.loc[plateau_peak_idx]
        ax.plot(plateau_peak_point["adjusted_frame"], plateau_peak_point["distance"], 'go', label='Plateau/Peak')
        #ax.set_xlim(plateau_peak_point["adjusted_frame"] - 20, plateau_peak_point["adjusted_frame"] + 20)
        
        ax.legend()
    except KeyError:
        print(f"Warning: No data found for event {event}")

# Adjust layout and save the plot
plt.tight_layout()
plt.savefig("/mnt/upramdya_data/MD/MultiMazeRecorder/Plots/250218_Refined_contacts/events_distance_withmarkers_and_plateaus_New.png")
plt.show()


In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import sys
from pathlib import Path

# Load the video
flypath = exfly["flypath"].iloc[0]
# Find the mp4 file in the folder
video_folder = Path(flypath)
video_files = list(video_folder.glob("*.mp4"))

video_path = video_files[0]
print(video_path)

cap = cv2.VideoCapture(str(video_path))

# Check if the video is opened
if not cap.isOpened():
    print("Error: Could not open video")
    sys.exit(1)

# Store frames in a list
frames = []

# For each event, get the frame corresponding to the detected start point
for event in exfly["interaction_event"].unique():
    # Get the start point for the event
    start_point = exfly.loc[plateau_or_peak[event]]
    
    # Set the frame number to the start point
    cap.set(cv2.CAP_PROP_POS_FRAMES, start_point["frame"])
    
    # Read the frame
    ret, frame = cap.read()
    
    # Check if the frame is read correctly
    if not ret:
        print(f"Error: Could not read frame for event {event}")
        continue
    
    # Convert the frame from BGR to RGB
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Append the frame to the list
    frames.append((event, frame))

# Release the video capture object
cap.release()

# Plot the frames in a grid
n_frames = len(frames)
n_cols = 5
n_rows = int(np.ceil(n_frames / n_cols))
fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 3 * n_rows))

for i, (event, frame) in enumerate(frames):
    row = i // n_cols
    col = i % n_cols
    ax = axes[row, col]
    ax.imshow(frame)
    ax.set_title(f"Event: {event}")
    ax.axis('off')

# Remove any empty subplots
for j in range(i + 1, n_rows * n_cols):
    fig.delaxes(axes.flatten()[j])

plt.tight_layout()
plt.show()

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import sys
from pathlib import Path

# Load the video
flypath = exfly["flypath"].iloc[0]
# Find the mp4 file in the folder
video_folder = Path(flypath)
video_files = list(video_folder.glob("*.mp4"))

if not video_files:
    print("Error: No mp4 files found in the folder")
    sys.exit(1)

video_path = video_files[0]
print(video_path)

cap = cv2.VideoCapture(str(video_path))

# Check if the video is opened
if not cap.isOpened():
    print("Error: Could not open video")
    sys.exit(1)

# Parameters
fps = 5
frames_before = 20
frames_after = 20
total_frames = frames_before + frames_after + 1
output_width = 3840
output_height = 2160

# Store frames in a dictionary
frames_dict = {event: [] for event in exfly["interaction_event"].unique()}

# For each event, get the frames around the detected start point
for event in exfly["interaction_event"].unique():
    # Get the start point for the event
    start_point = exfly.loc[plateau_or_peak[event]]["frame"]
    
    for i in range(-frames_before, frames_after + 1):
        frame_number = start_point + i
        # Set the frame number
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
        
        # Read the frame
        ret, frame = cap.read()
        
        # Check if the frame is read correctly
        if not ret:
            print(f"Error: Could not read frame {frame_number} for event {event}")
            continue
        
        # Convert the frame from BGR to RGB
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Append the frame to the list
        frames_dict[event].append(frame)

# Release the video capture object
cap.release()

# Create a video writer
output_video_path = "grid_video.mp4"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
frame_height, frame_width, _ = next(iter(frames_dict.values()))[0].shape
# Parameters (adjust as needed)
# Modify the grid calculation section with this optimized version
n_events = len(frames_dict)
frame_height, frame_width, _ = next(iter(frames_dict.values()))[0].shape
aspect_ratio = frame_width / frame_height

# Calculate optimal grid dimensions for 16:9
target_aspect = 16/9
approx_cols = np.sqrt((target_aspect / aspect_ratio) * n_events)
candidates = [int(np.floor(approx_cols)), int(np.ceil(approx_cols)), int(round(approx_cols))]
candidates = np.unique(np.clip(candidates, 1, n_events))

best_cols = 1
best_error = float('inf')

for cols in candidates:
    rows = np.ceil(n_events / cols).astype(int)
    grid_ar = (cols * aspect_ratio) / rows  # Combined grid aspect ratio
    error = abs(grid_ar - target_aspect)
    if error < best_error:
        best_error = error
        best_cols = cols

n_cols = best_cols
n_rows = np.ceil(n_events / n_cols).astype(int)

# Set output resolution to 4K
output_width = 3840
output_height = 2160

# Calculate grid dimensions
grid_width = frame_width * n_cols
grid_height = frame_height * n_rows

# Maintain aspect ratio in scaling
scaling_factor = min(output_width/grid_width, output_height/grid_height)
resized_width = int(grid_width * scaling_factor)
resized_height = int(grid_height * scaling_factor)

# Initialize video writer with 4K settings
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Better for 4K
out = cv2.VideoWriter(str(output_video_path), fourcc, fps, 
                     (resized_width, resized_height))

# Grid composition with optimized layout
for i in range(total_frames):
    grid_frame = np.zeros((grid_height, grid_width, 3), dtype=np.uint8)
    
    for event_idx, (event, frames) in enumerate(frames_dict.items()):
        row = event_idx // n_cols
        col = event_idx % n_cols
        
        if i < len(frames):
            x = col * frame_width
            y = row * frame_height
            grid_frame[y:y+frame_height, x:x+frame_width] = frames[i]
    
    # High-quality scaling for 4K
    resized_frame = cv2.resize(grid_frame, (resized_width, resized_height), 
                              interpolation=cv2.INTER_LANCZOS4)
    resized_frame = cv2.cvtColor(resized_frame, cv2.COLOR_RGB2BGR)
    out.write(resized_frame)

out.release()


print(f"Grid video saved to {output_video_path}")

In [ ]:
# Select event 65 and try different parameters for peak / plateau detection
event_data = exfly.loc[exfly["interaction_event"] == 94]

# Create a single plot
plt.figure(figsize=(10, 6))
plt.plot(event_data["frame"], event_data["distance"], label='Distance')

# Try different parameters for peak / plateau detection

# Default parameters
start_idx = find_interaction_start(event_data, 'distance', 'frame')
plt.axvline(event_data.loc[start_idx, 'frame'], color='r', label='Default')

# Increase window size
start_idx = find_interaction_start(event_data, 'distance', 'frame', window_size=50)
plt.axvline(event_data.loc[start_idx, 'frame'], color='orange', label='Window Size 7')

# Change peak window size

start_idx = find_interaction_start(event_data, 'distance', 'frame', peak_window_size=10)
plt.axvline(event_data.loc[start_idx, 'frame'], color='blue', label='Peak Window Size 10')

# Increase minimum plateau length
start_idx = find_interaction_start(event_data, 'distance', 'frame', min_plateau_length=40)
plt.axvline(event_data.loc[start_idx, 'frame'], color='green', label='Min Plateau Length 10')

# Increase peak prominence
start_idx = find_interaction_start(event_data, 'distance', 'frame', peak_prominence=1)
plt.axvline(event_data.loc[start_idx, 'frame'], color='purple', label='Peak Prominence 2.0')

# Increase dynamic threshold multiplier
start_idx = find_interaction_start(event_data, 'distance', 'frame', threshold_multiplier=3)
plt.axvline(event_data.loc[start_idx, 'frame'], color='brown', label='Threshold Multiplier 3')

# Add legend and show plot
plt.legend()
plt.xlabel('Frame')
plt.ylabel('Distance')
plt.title('Event 65: Distance with Different Parameters for Peak/Plateau Detection')
plt.show()

In [ ]:
# Pick another fly to double check the detection code

exfly = ball_data.loc[ball_data["fly"] == ball_data["fly"].unique()[19]]

# for each event, compute the distance between the fly and the ball
for event in exfly["interaction_event"].unique():
    event_data = exfly.loc[exfly["interaction_event"] == event]
    event_data = event_data.sort_values("frame")
    event_data["distance"] = np.sqrt(
        (event_data["x_ball_0"] - event_data["x_fly_0"]) ** 2
        + (event_data["y_ball_0"] - event_data["y_fly_0"]) ** 2
    )
    
    # Add an "adjusted_frame" column that starts at 0 and goes up to the length of the event
    event_data["adjusted_frame"] = range(len(event_data))

    # Assign the computed columns back to the original DataFrame
    exfly.loc[exfly["interaction_event"] == event, "distance"] = event_data["distance"]
    exfly.loc[exfly["interaction_event"] == event, "adjusted_frame"] = event_data["adjusted_frame"]

print(exfly)
# Find first minimum distance for each event
min_distance = exfly.groupby("interaction_event")["distance"].min()
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.signal import find_peaks, savgol_filter

def find_interaction_start(data, distance_col, frame_col, 
                          threshold_multiplier=1.5, window_size=30, 
                          min_plateau_length=50, peak_prominence=1,
                          peak_window_size=10):
    # Work on a copy to avoid modifying original data
    data = data.copy()
    
    # Smoothing and derivative calculation
    data['smoothed_distance'] = savgol_filter(data[distance_col], 
                                            window_length=11, 
                                            polyorder=3)
    data['smoothed_diff'] = data['smoothed_distance'].diff()
    data['smoothed_accel'] = data['smoothed_diff'].diff()  # Second derivative

    # Dynamic threshold for plateaus based on rolling standard deviation
    rolling_std = data['smoothed_diff'].rolling(window=window_size).std()
    dynamic_threshold = rolling_std.mean() * threshold_multiplier  # Adjust multiplier as needed

    # Plateau detection with dynamic threshold
    plateau_mask = (data['smoothed_diff'].abs() < dynamic_threshold)
    plateau_groups = (plateau_mask != plateau_mask.shift()).cumsum()
    
    # Initialize plateau markers
    data['plateau_start'] = 0
    valid_plateaus = data[plateau_mask].groupby(plateau_groups).filter(
        lambda x: len(x) >= min_plateau_length
    )
    if not valid_plateaus.empty:
        start_indices = valid_plateaus.groupby(plateau_groups).head(1).index
        data.loc[start_indices, 'plateau_start'] = 1

    # Peak detection with stricter prominence
    peaks, _ = find_peaks(-data['smoothed_distance'],
                        prominence=peak_prominence,
                        width=3)
    
    # Refine peak detection for better alignment
    refined_peaks = []
    for peak in peaks:
        if peak > 0 and peak < len(data) - 1:
            # Perform local search around the detected peak to refine position
            local_region = data.iloc[max(0, peak - peak_window_size):min(len(data), peak + peak_window_size)]
            true_peak_idx = local_region[distance_col].idxmin()  # Find true minimum in this region
            refined_peaks.append(true_peak_idx)

    # Combine plateau and refined peak detections
    plateau_indices = data[data['plateau_start'] == 1].index
    all_candidates = sorted(list(plateau_indices) + refined_peaks)
    
    # Fallback to minimum distance if no markers found
    if not all_candidates:
        return data[distance_col].idxmin()
    
    return all_candidates[0]


# Filter out NaN values from interaction_event
exfly = exfly.dropna(subset=['interaction_event'])

# Convert interaction_event to integer sequence first
exfly['interaction_event'] = exfly['interaction_event'].astype(int)



# Find first minimum distance and plateau/peak for each event
min_distance = exfly.groupby("interaction_event")["distance"].idxmin()
plateau_or_peak = exfly.groupby("interaction_event").apply(
    lambda x: find_interaction_start(x, 'distance', 'adjusted_frame',
                                    )
)

# Create a grid
grid = sns.FacetGrid(exfly, col="interaction_event", col_wrap=4)

# Plot the distance for each event
grid.map(plt.plot, "adjusted_frame", "distance")

# Add markers for the minimum distance points and plateau/peak points
for ax, event in zip(grid.axes.flat, exfly["interaction_event"].unique()):
    try:
        # Plot minimum distance point
        min_idx = min_distance[event]
        min_point = exfly.loc[min_idx]
        ax.plot(min_point["adjusted_frame"], min_point["distance"], 'ro', label='Minimum')
        
        # Plot plateau/peak point
        plateau_peak_idx = plateau_or_peak[event]
        plateau_peak_point = exfly.loc[plateau_peak_idx]
        ax.plot(plateau_peak_point["adjusted_frame"], plateau_peak_point["distance"], 'go', label='Plateau/Peak')
        #ax.set_xlim(plateau_peak_point["adjusted_frame"] - 20, plateau_peak_point["adjusted_frame"] + 20)
        
        ax.legend()
    except KeyError:
        print(f"Warning: No data found for event {event}")

# Adjust layout and save the plot
plt.tight_layout()
plt.savefig("/mnt/upramdya_data/MD/MultiMazeRecorder/Plots/250218_Refined_contacts/events_distance_withmarkers_and_plateaus_New_SecondFly.png")
plt.show()


In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import sys
from pathlib import Path

# Load the video
flypath = exfly["flypath"].iloc[0]
# Find the mp4 file in the folder
video_folder = Path(flypath)
video_files = list(video_folder.glob("*.mp4"))

video_path = video_files[0]
print(video_path)

cap = cv2.VideoCapture(str(video_path))

# Check if the video is opened
if not cap.isOpened():
    print("Error: Could not open video")
    sys.exit(1)

# Store frames in a list
frames = []

# For each event, get the frame corresponding to the detected start point
for event in exfly["interaction_event"].unique():
    # Get the start point for the event
    start_point = exfly.loc[plateau_or_peak[event]]
    
    # Set the frame number to the start point
    cap.set(cv2.CAP_PROP_POS_FRAMES, start_point["frame"])
    
    # Read the frame
    ret, frame = cap.read()
    
    # Check if the frame is read correctly
    if not ret:
        print(f"Error: Could not read frame for event {event}")
        continue
    
    # Convert the frame from BGR to RGB
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Append the frame to the list
    frames.append((event, frame))

# Release the video capture object
cap.release()

# Plot the frames in a grid
n_frames = len(frames)
n_cols = 5
n_rows = int(np.ceil(n_frames / n_cols))
fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 3 * n_rows))

for i, (event, frame) in enumerate(frames):
    row = i // n_cols
    col = i % n_cols
    ax = axes[row, col]
    ax.imshow(frame)
    ax.set_title(f"Event: {event}")
    ax.axis('off')

# Remove any empty subplots
for j in range(i + 1, n_rows * n_cols):
    fig.delaxes(axes.flatten()[j])

plt.tight_layout()
plt.show()

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import sys
from pathlib import Path

# Load the video
flypath = exfly["flypath"].iloc[0]
# Find the mp4 file in the folder
video_folder = Path(flypath)
video_files = list(video_folder.glob("*.mp4"))

if not video_files:
    print("Error: No mp4 files found in the folder")
    sys.exit(1)

video_path = video_files[0]
print(video_path)

cap = cv2.VideoCapture(str(video_path))

# Check if the video is opened
if not cap.isOpened():
    print("Error: Could not open video")
    sys.exit(1)

# Parameters
fps = 5
frames_before = 20
frames_after = 20
total_frames = frames_before + frames_after + 1
output_width = 3840
output_height = 2160

# Store frames in a dictionary
frames_dict = {event: [] for event in exfly["interaction_event"].unique()}

# For each event, get the frames around the detected start point
for event in exfly["interaction_event"].unique():
    # Get the start point for the event
    start_point = exfly.loc[plateau_or_peak[event]]["frame"]
    
    for i in range(-frames_before, frames_after + 1):
        frame_number = start_point + i
        # Set the frame number
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
        
        # Read the frame
        ret, frame = cap.read()
        
        # Check if the frame is read correctly
        if not ret:
            print(f"Error: Could not read frame {frame_number} for event {event}")
            continue
        
        # Convert the frame from BGR to RGB
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Append the frame to the list
        frames_dict[event].append(frame)

# Release the video capture object
cap.release()

# Create a video writer
output_video_path = "grid_video_New.mp4"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
frame_height, frame_width, _ = next(iter(frames_dict.values()))[0].shape
# Parameters (adjust as needed)
# Modify the grid calculation section with this optimized version
n_events = len(frames_dict)
frame_height, frame_width, _ = next(iter(frames_dict.values()))[0].shape
aspect_ratio = frame_width / frame_height

# Calculate optimal grid dimensions for 16:9
target_aspect = 16/9
approx_cols = np.sqrt((target_aspect / aspect_ratio) * n_events)
candidates = [int(np.floor(approx_cols)), int(np.ceil(approx_cols)), int(round(approx_cols))]
candidates = np.unique(np.clip(candidates, 1, n_events))

best_cols = 1
best_error = float('inf')

for cols in candidates:
    rows = np.ceil(n_events / cols).astype(int)
    grid_ar = (cols * aspect_ratio) / rows  # Combined grid aspect ratio
    error = abs(grid_ar - target_aspect)
    if error < best_error:
        best_error = error
        best_cols = cols

n_cols = best_cols
n_rows = np.ceil(n_events / n_cols).astype(int)

# Set output resolution to 4K
output_width = 3840
output_height = 2160

# Calculate grid dimensions
grid_width = frame_width * n_cols
grid_height = frame_height * n_rows

# Maintain aspect ratio in scaling
scaling_factor = min(output_width/grid_width, output_height/grid_height)
resized_width = int(grid_width * scaling_factor)
resized_height = int(grid_height * scaling_factor)

# Initialize video writer with 4K settings
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Better for 4K
out = cv2.VideoWriter(str(output_video_path), fourcc, fps, 
                     (resized_width, resized_height))

# Grid composition with optimized layout
for i in range(total_frames):
    grid_frame = np.zeros((grid_height, grid_width, 3), dtype=np.uint8)
    
    for event_idx, (event, frames) in enumerate(frames_dict.items()):
        row = event_idx // n_cols
        col = event_idx % n_cols
        
        if i < len(frames):
            x = col * frame_width
            y = row * frame_height
            grid_frame[y:y+frame_height, x:x+frame_width] = frames[i]
    
    # High-quality scaling for 4K
    resized_frame = cv2.resize(grid_frame, (resized_width, resized_height), 
                              interpolation=cv2.INTER_LANCZOS4)
    resized_frame = cv2.cvtColor(resized_frame, cv2.COLOR_RGB2BGR)
    out.write(resized_frame)

out.release()


print(f"Grid video saved to {output_video_path}")

In [ ]:
# Now let's crop the interaction events to 20 frames before and after the minimum distance point and plot the grid again

# Create a grid

grid = sns.FacetGrid(exfly, col="interaction_event", col_wrap=4)

# Plot the distance for each event

grid.map(plt.plot, "adjusted_frame", "distance")

# Add markers for the minimum distance points

for ax, event in zip(grid.axes.flat, exfly["interaction_event"].unique()):
    
    try:
        min_idx = min_distance[event]
        min_point = exfly.loc[min_idx]
        ax.plot(min_point["adjusted_frame"], min_point["distance"], 'ro')
        ax.set_xlim(min_point["adjusted_frame"] - 20, min_point["adjusted_frame"] + 20)
    except KeyError:
        print(f"Warning: No minimum distance found for event {event}")
        
# Save the plot

plt.savefig("/mnt/upramdya_data/MD/MultiMazeRecorder/Plots/250218_Refined_contacts/events_distance_withmarkers_cropped.png")

plt.show()

In [ ]:
# Load contact data 

Contact_data = Ballpushing_utils.Dataset(exp, dataset_type="contact_data")

In [ ]:
Contact_data.data

In [ ]:
Contact_data.data.columns

In [ ]:
# Make a column with the distance between the fly and the ball

Contact_data.data["distance"] = np.sqrt(
    (Contact_data.data["x_centre_preprocessed"] - Contact_data.data["x_Thorax"]) ** 2
    + (Contact_data.data["y_centre_preprocessed"] - Contact_data.data["y_Thorax"]) ** 2
)

In [ ]:
# Make a "adjusted_frame" column that starts at 0 and goes up to the length of the event. These should be computed grouped by contact_index

Contact_data.data["adjusted_frame"] = Contact_data.data.groupby("contact_index")["frame"].transform(
    lambda x: x - x.min()
)

In [ ]:
# Plot a grid and for each event plot the curve of the distance between the fly and the ball during the event

# Create a grid

grid = sns.FacetGrid(Contact_data.data, col="contact_index", col_wrap=4)

# Plot the distance for each event

grid.map(plt.plot, "adjusted_frame", "distance")
